In [1]:
# STEP 1: Install dependencies
!pip install ta requests --quiet

  Preparing metadata (setup.py) ... done


In [5]:
# ✅ BTC Price Prediction using GRU + Technical Indicators + CoinGecko

# STEP 1: Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import requests
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense
from tensorflow.keras.callbacks import EarlyStopping

# STEP 2: Load dataset (filtered 2023+ CSV)
df = pd.read_csv('/content/btc_1h_data_2018_to_2025.csv')
df['Open time'] = pd.to_datetime(df['Open time'])
df = df[df['Open time'] >= '2023-01-01']

# STEP 3: Add technical indicators manually
df['EMA_20'] = df['Close'].ewm(span=20, adjust=False).mean()
delta = df['Close'].diff()
gain = delta.where(delta > 0, 0)
loss = -delta.where(delta < 0, 0)
avg_gain = gain.rolling(window=14).mean()
avg_loss = loss.rolling(window=14).mean()
rs = avg_gain / avg_loss
df['RSI_14'] = 100 - (100 / (1 + rs))
df['MACD'] = df['Close'].ewm(span=12, adjust=False).mean() - df['Close'].ewm(span=26, adjust=False).mean()
df.dropna(inplace=True)

# STEP 4: Feature selection + scaling
features = ['Open', 'High', 'Low', 'Close', 'Volume', 'EMA_20', 'RSI_14', 'MACD']
data = df[features]
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(data)

# STEP 5: Sequence generation (30-hour window)
def create_sequences(data, seq_len=30):
    X, y = [], []
    for i in range(len(data) - seq_len):
        X.append(data[i:i+seq_len])
        y.append(data[i+seq_len][3])  # 'Close'
    return np.array(X), np.array(y)

X, y = create_sequences(scaled_data)
split = int(0.8 * len(X))
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

# STEP 6: Build and train GRU model
model = Sequential([
    GRU(64, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])),
    GRU(64),
    Dense(32),
    Dense(1)
])

model.compile(optimizer='adam', loss='mean_squared_error')
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=50,
    batch_size=32,
    callbacks=[early_stop],
    verbose=1
)

# STEP 7: Evaluate model
predictions = model.predict(X_test)
pred_scaled = np.zeros((len(predictions), scaled_data.shape[1]))
pred_scaled[:, 3] = predictions[:, 0]
pred_unscaled = scaler.inverse_transform(pred_scaled)[:, 3]

actual_scaled = np.zeros((len(y_test), scaled_data.shape[1]))
actual_scaled[:, 3] = y_test
actual_unscaled = scaler.inverse_transform(actual_scaled)[:, 3]

rmse = np.sqrt(mean_squared_error(actual_unscaled, pred_unscaled))
mae = mean_absolute_error(actual_unscaled, pred_unscaled)
r2 = r2_score(actual_unscaled, pred_unscaled)

print(f"\n✅ RMSE: {rmse:.2f}\n✅ MAE: {mae:.2f}\n✅ R²: {r2 * 100:.2f}%")

# STEP 8: Predict next price
last_seq = scaled_data[-30:].reshape(1, 30, len(features))
next_close_scaled = model.predict(last_seq)[0][0]
next_point = np.zeros((1, scaled_data.shape[1]))
next_point[0][3] = next_close_scaled
next_close_price = scaler.inverse_transform(next_point)[0][3]
print(f"\n🔮 Predicted Next Close: ${next_close_price:.2f}")

# STEP 9: Get live BTC price from CoinGecko
url = 'https://api.coingecko.com/api/v3/simple/price?ids=bitcoin&vs_currencies=usd'
live_price = requests.get(url).json()['bitcoin']['usd']
print(f"📈 Live BTC Price: ${live_price:.2f}")

# STEP 10: Candlestick chart with predicted point
plot_df = df[-50:].copy()
plot_df['Predicted Close'] = np.nan
plot_df.iloc[-1, plot_df.columns.get_loc('Predicted Close')] = next_close_price

fig = go.Figure(data=[
    go.Candlestick(
        x=plot_df['Open time'],
        open=plot_df['Open'], high=plot_df['High'],
        low=plot_df['Low'], close=plot_df['Close'],
        name='Candlestick'
    ),
    go.Scatter(
        x=plot_df['Open time'],
        y=plot_df['Predicted Close'],
        mode='markers+lines', name='Predicted Close',
        line=dict(color='red', dash='dot'), marker=dict(size=10)
    )
])

fig.update_layout(title='BTC Price with Predicted Close', xaxis_title='Time', yaxis_title='Price (USD)', xaxis_rangeslider_visible=False)
fig.show()


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



Epoch 1/50
520/520 ━━━━━━━━━━━━━━━━━━━━ 26s 42ms/step - loss: 0.0046 - val_loss: 1.2869e-04
Epoch 2/50
520/520 ━━━━━━━━━━━━━━━━━━━━ 40s 40ms/step - loss: 1.7068e-05 - val_loss: 1.3580e-04
Epoch 3/50
520/520 ━━━━━━━━━━━━━━━━━━━━ 40s 38ms/step - loss: 1.7258e-05 - val_loss: 4.8975e-05
Epoch 4/50
520/520 ━━━━━━━━━━━━━━━━━━━━ 22s 40ms/step - loss: 1.9136e-05 - val_loss: 4.1813e-05
Epoch 5/50
520/520 ━━━━━━━━━━━━━━━━━━━━ 40s 39ms/step - loss: 1.4943e-05 - val_loss: 4.1776e-05
Epoch 6/50
520/520 ━━━━━━━━━━━━━━━━━━━━ 22s 41ms/step - loss: 1.8637e-05 - val_loss: 1.9863e-04
Epoch 7/50
520/520 ━━━━━━━━━━━━━━━━━━━━ 40s 39ms/step - loss: 1.9791e-05 - val_loss: 4.0439e-05
Epoch 8/50
520/520 ━━━━━━━━━━━━━━━━━━━━ 24s 47ms/step - loss: 1.9601e-05 - val_loss: 1.4619e-04
Epoch 9/50
520/520 ━━━━━━━━━━━━━━━━━━━━ 37s 39ms/step - loss: 2.9174e-05 - val_loss: 5.6601e-05
Epoch 10/50
520/520 ━━━━━━━━━━━━━━━━━━━━ 40s 37ms/step - loss: 1.5660e-05 - val_loss: 1.2233e-04
Epoch 11/50
520/520 ━━━━━━━━━━━━━━━━━━━━ 21

In [8]:
# === Predict next 3 closing prices ===
future_steps = 3
future_predictions = []

last_seq = scaled_data[-30:].copy()

for _ in range(future_steps):
    input_seq = last_seq.reshape(1, 30, len(features))
    pred = model.predict(input_seq)[0][0]

    # Start from the last real known point
    new_point = last_seq[-1].copy()  # last known state
    new_point[3] = pred  # update only 'Close'

    # Append prediction for inverse scaling
    future_scaled = new_point.reshape(1, -1)
    future_unscaled = scaler.inverse_transform(future_scaled)[0][3]
    future_predictions.append(future_unscaled)

    # Append predicted point to sequence (simulate next hour)
    last_seq = np.vstack([last_seq[1:], new_point])

# === Print results ===
print("\n🔮 Predicted BTC Closing Prices for Next 3 Hours:")
for i, price in enumerate(future_predictions, 1):
    print(f"Hour +{i}: ${price:.2f}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step

🔮 Predicted BTC Closing Prices for Next 3 Hours:
Hour +1: $103181.87
Hour +2: $102985.38
Hour +3: $102901.08


In [10]:
import plotly.graph_objects as go
from datetime import timedelta

# Get the last real timestamp
last_time = df['Open time'].iloc[-1]

# Generate dummy candles from predicted prices
predicted_candles = []

prev_close = df['Close'].iloc[-1]
for i, predicted_close in enumerate(future_predictions):
    open_price = prev_close
    close_price = predicted_close
    high_price = max(open_price, close_price) * 1.005  # +0.5% buffer
    low_price = min(open_price, close_price) * 0.995  # -0.5% buffer
    time = last_time + timedelta(hours=i + 1)

    predicted_candles.append({
        'Open time': time,
        'Open': open_price,
        'High': high_price,
        'Low': low_price,
        'Close': close_price
    })
    prev_close = close_price  # chain the candles

# Convert to DataFrame
pred_df = pd.DataFrame(predicted_candles)

# Plot actual last 30 + predicted 3
plot_df = pd.concat([df[['Open time', 'Open', 'High', 'Low', 'Close']].iloc[-30:], pred_df])

fig = go.Figure(data=[
    go.Candlestick(
        x=plot_df['Open time'],
        open=plot_df['Open'],
        high=plot_df['High'],
        low=plot_df['Low'],
        close=plot_df['Close'],
        name="BTC Price"
    )
])

fig.update_layout(
    title="📉 BTC Price with Predicted 3-Hour Candles",
    xaxis_title="Time",
    yaxis_title="Price (USD)",
    xaxis_rangeslider_visible=False
)
fig.show()
